<a href="https://colab.research.google.com/github/Gus-1003/ISD_psb20232/blob/main/Machine_Learning/EEG_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Bibliotecas:

## Gerais:

In [27]:
import os
import random
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
from mpl_toolkits.mplot3d import Axes3D

> import os: O módulo os é essencial para interagir com o sistema operacional. Ele permite que você acesse e manipule arquivos e pastas, execute comandos do sistema e obtenha informações sobre o ambiente de execução.

> import random: O módulo random fornece funcionalidades para a geração de números aleatórios. Isso é útil em uma variedade de aplicações, como simulações, jogos e testes.

> import pandas as pd: O Pandas é uma biblioteca amplamente usada para manipulação e análise de dados em Python. Ao importar o Pandas como pd, você pode usar suas funções para carregar, manipular e analisar dados tabulares de forma eficaz.

> import numpy as np: O NumPy é uma biblioteca fundamental para computação científica em Python. Ele fornece suporte para arrays multidimensionais e funções para operações matemáticas e científicas eficientes. Importar o NumPy como np é uma convenção comum.

> import math: O módulo math oferece funções matemáticas padrão, como seno, cosseno, logaritmos, etc. Ele é útil para operações matemáticas avançadas em seu código.

> import matplotlib.pyplot as plt: O Matplotlib é uma biblioteca gráfica amplamente usada para criar visualizações de dados. Importar o Matplotlib como plt permite que você crie gráficos, gráficos e plotagens de maneira conveniente.

> import pickle: O módulo pickle permite a serialização e desserialização de objetos Python. Isso é útil para armazenar objetos em arquivos ou transmiti-los pela rede.

> from mpl_toolkits.mplot3d import Axes3D: Esta linha importa a classe Axes3D do módulo mpl_toolkits.mplot3d. Isso é útil quando você deseja criar gráficos 3D usando o Matplotlib, permitindo que você crie visualizações tridimensionais.



## Modulos do Scipy:

In [28]:
import scipy.fftpack
from scipy.fft import fft, ifft
from scipy.spatial import distance # Interessante - Talvez dê para aplicar a imagens
from scipy.spatial.distance import euclidean
from scipy.interpolate import interp1d
from scipy.signal import periodogram

> import scipy.fftpack: O SciPy é uma biblioteca que amplia as capacidades do NumPy, fornecendo funções adicionais para análise numérica e científica. O scipy.fftpack é uma subbiblioteca para transformadas de Fourier, útil na análise de séries temporais e sinais.

> from scipy.fft import fft, ifft: O scipy.fft é um módulo que fornece funções para calcular a transformada rápida de Fourier (FFT) e a transformada inversa de Fourier (IFFT). Essas funções são cruciais para análise de sinais e processamento de frequência.

> from scipy.spatial import distance: O scipy.spatial é um submódulo que contém funções para calcular distâncias entre pontos no espaço multidimensional. Isso é útil em tarefas como clustering, classificação e análise de dados espaciais.

> from scipy.spatial.distance import euclidean: A importação específica da função euclidean do módulo scipy.spatial.distance permite calcular a distância euclidiana entre dois pontos em um espaço multidimensional. A distância euclidiana é uma medida comum de distância entre pontos em geometria.

> from scipy.interpolate import interp1d: O scipy.interpolate é usado para interpolação de dados. A função interp1d permite criar uma função interpolante a partir de dados discretos, o que é útil para preencher espaços entre pontos de dados e fazer estimativas suaves.

> from scipy.signal import periodogram: O scipy.signal fornece funções para processamento de sinais, incluindo a estimativa do espectro de potência de um sinal. A função periodogram é usada para calcular o espectro de potência de um sinal, o que é importante em análise de sinais e frequências.

## Modulos do sklearn:

In [29]:
from sklearn import *

In [30]:
from sklearn.preprocessing import StandardScaler

In [31]:
from sklearn.decomposition import PCA

In [32]:
from sklearn.utils import resample

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [34]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [35]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import confusion_matrix

In [36]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

# Funções:

In [37]:
def band_pass_filter(signal, fs, f_low, f_high):
    m, n = signal.shape
    freqs = np.fft.fftfreq(m, 1 / fs)
    signal_fft = np.fft.fft(signal, axis=0)
    mask = (freqs >= f_low) & (freqs <= f_high)
    filtered_signal_fft = signal_fft * mask[:, np.newaxis]
    filtered_signal = 2*np.fft.ifft(filtered_signal_fft, axis=0)
    return np.real(filtered_signal)

def CAR_filter(dataframe_each_posture):
    wheelchair_filtered2 = []
    wheelchair_filtered2list = []
    tot=[]
    list_help=[]

    for i in range (dataframe_each_posture.shape[0]): #mean each row of the dataframe
      wheelchair_filtered2.append(dataframe_each_posture.iloc[i].mean())

    for i in range (dataframe_each_posture.shape[0]):   #Loop through N columns of the dataframe

      for j in dataframe_each_posture.iloc[i]:          #Loop throug M rows of the dataframe
        list_help.append(j-wheelchair_filtered2[i])
      tot.append(list_help)
      list_help=[]

    posture_dffilt = pd.DataFrame(tot)  #Dataframe filtered

    #columns_PDs=['PDseat11','PDseat12','PDseat21','PDseat22','PDseat31','PDseat32','PDseat41','PDseat42','PDseat51','PDseat52','PDseat61','PDseat62','PDback11','PDback12','PDback21','PDback22','PDback31','PDback32','PDback41','PDback42','PDback51','PDback52','PDback61','PDback62']#,'Control_LEDs']
    #posture_dffilt.columns = columns_PDs

    return posture_dffilt

def features(df):
    mu_min_freq = 8  # Hz
    mu_max_freq = 13  # Hz
    beta_max_freq = 20

    dfs=list()

    # Aplicar el periodograma
    for i in (df.columns):
        fft_df = np.fft.fft(df[i].values)

        freqs=np.fft.fftfreq(len(df[i].values),1/fs_eeg)

        powfft=np.abs(fft_df)**2

        pow_range_mu = np.mean(powfft[np.where((freqs >= 8) & (freqs <= 13))])

        pow_range_beta = np.mean(powfft[np.where((freqs >= 13) & (freqs <= 20))])

        #print("pow_range_mu: ", pow_range_mu, "pow_range_beta: ",pow_range_beta)

        powboth=[pow_range_mu,pow_range_beta]

        dfs.append(pow_range_mu)

        dfs.append(pow_range_beta)

    #print("dfs: ",dfs,len(dfs))
    return dfs

def windows_generation_overlap(dataframe_filtered,lab):
    #Lists to store windows of data
    W = []
    featwind = []
    #print("shape X: ",X.shape[0])
    windows_quantity = int(((dataframe_filtered.shape[0] - samples) / overlap) - 1)  #Definition of number of windows
    var = 0
    W = [dataframe_filtered.iloc[var:samples - 1]]
    featwindows = list()
    for i in range(windows_quantity):   #Windowing considering overlap
        var += overlap
        W.append(dataframe_filtered.iloc[var:(var + samples) - 1])
    #print("# windows: ",windows_quantity,"W: ",len(W),len(W[0]))
    #print("df per window: ",W[0])
    for i in range(len(W)):  # Window 0 to N window
        #print("window n: ",i,len(W))
        dffeatures = pd.DataFrame(W[i])
        #print("dffeatures: ",dffeatures)
        #print("labwind: ",labwind.shape)
        #Raw data organized per window and overlap
        filtered_eegclass = band_pass_filter(dffeatures, fs_eeg, 8, 30)
        df_filtered_eegclass = pd.DataFrame(filtered_eegclass)
        CAR_eegclass = CAR_filter(df_filtered_eegclass)
        #print("filtered dffeatures: ", CAR_eegclass)
        featdf = features(CAR_eegclass)
        featwind.append(featdf)
        featdf[-1] = lab
    df_all = pd.DataFrame(featwind)
    #print("shape all: ",df_all.shape)
    return df_all

def readingEEGdata(df_eeg):
    new_df = df_eeg.fillna(0)
    eeg_data = new_df.iloc[:,2:17].copy()
    Id =  new_df.iloc[:,18].copy()
    time = new_df.iloc[:,0].copy()

    close_idx = Id.index[Id == 786].tolist()
    open_idx = Id.index[(Id == 769) | (Id == 770)].tolist()
    base_idx = Id.index[(Id == 32775) | (Id == 32776)].tolist()
    end_idx = Id.index[Id == 800].tolist()

    columnlabclase1=list()
    columnlabclase2=list()
    columnlabclassess=list()
    #print(close_idx)
    #print("Number of trials: ",len(open_idx))
    featclasses=list()
    featlab = list()

    for i in range(len(open_idx)):
        clase1 = eeg_data.iloc[close_idx[i]:open_idx[i]]
        clase2 = eeg_data.iloc[open_idx[i]:end_idx[i]]
        #Unbalanced class
        clase1 = clase1.iloc[232:695]
        #print("clase1 shape: ",clase1.shape,"clase2 shape: ",clase2.shape)
        # Adding label to each class
        #featclase1 = windows_generation_overlap(clase1,1)
        #Lists to store windows of data
        W = []
        featwind = []
        windows_quantity = int(((clase1.shape[0] - samples) / overlap) - 1)  #Definition of number of windows
        var = 0
        W = [clase1.iloc[var:samples - 1]]
        featwindows = list()

        for i in range(windows_quantity):   #Windowing considering overlap
            var += overlap
            W.append(clase1.iloc[var:(var + samples) - 1])


        #print("# windows: ",windows_quantity,"W: ",len(W),len(W[0]))
        #print("df per window: ",W[0])

        for i in range(len(W)):  # Window 0 to N window

            #print("window n: ",i,len(W))

            dffeatures = pd.DataFrame(W[i])

            #print("dffeatures: ",dffeatures)
            #print("labwind: ",labwind.shape)
            #Raw data organized per window and overlap

            filtered_eegclass = band_pass_filter(dffeatures, fs_eeg, 8, 30)
            df_filtered_eegclass = pd.DataFrame(filtered_eegclass)
            CAR_eegclass = CAR_filter(df_filtered_eegclass)

            #print("filtered dffeatures: ", CAR_eegclass)

            featdf = features(CAR_eegclass)
            featwind.append(featdf)
            featdf[-1] = 1

        df_all = pd.DataFrame(featwind)
        #print("shape all: ",df_all.shape)
        featclase2 = windows_generation_overlap(clase2,2)

        df_classes = pd.concat([featclase1, featclase2], axis=0)
        #print("df_classes: ", df_classes)
        #print("df_classes: ",df_classes)
        featclasses.append(df_classes)

    df_totclasses = pd.concat(featclasses, axis=0)
    #df_totclasses['labels'] = columnlabclassess
    #df_totlabels= pd.concat(featlab, axis=0)
    #print("df_totclasses: ",df_totclasses)
    return df_totclasses

def min_max(column):
    min_value = column.min()
    max_value = column.max()
    all_min.append(min_value)
    all_max.append(max_value)
    min_max= c + ((d-c)*(column - min_value) / (max_value - min_value))
    return min_max


# Codigo Principal:

## Leitura de CSV's:

In [38]:
sess1 = pd.read_csv("/content/sess1.csv")
sess2 = pd.read_csv("/content/sess2.csv")
sess3 = pd.read_csv("/content/sess3.csv")
sess4 = pd.read_csv("/content/sess4.csv")
sess5 = pd.read_csv("/content/sess5.csv")

## Variáveis Globais:

In [39]:
num_segments = 10
samples = 125
overlap = 62
fs_eeg = 125
all_min = list()
all_max = list()
c = -1
d = 1

## Pré-preocessamento:

### Divisão Treino - Teste:

In [40]:
#Split of sessions for train, test and validation
df_eeg_train = pd.concat([sess3,sess1,sess5,sess4])
df_eeg_val = sess2

In [41]:
#EEG reading data and cross-trials validation
EEG_train = readingEEGdata(df_eeg_train)

n_splits = 5

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

trialstrain=list()
trialstest=list()
labtrain=list()
labtest=list()

for train_index, test_index in kf.split(EEG_train):
    feattr, featte = EEG_train.iloc[train_index], EEG_train.iloc[test_index]

    X_train, y_train = feattr.iloc[:, :-1], feattr.iloc[:,-1:]
    X_test, y_test = featte.iloc[:, :-1], featte.iloc[:,-1:]

    # Normalization for train and test
    X_train = X_train.apply(min_max)

    for column in X_test.columns:
        # print("test: ",all_min[X_test.columns.get_loc(column)],all_max[X_test.columns.get_loc(column)],X_test[column])
        X_test[column] = c + ((d - c) * (X_test[column] - all_min[X_test.columns.get_loc(column)]) / (
                    all_max[X_test.columns.get_loc(column)] - all_min[X_test.columns.get_loc(column)]))

    #print("X shape: ",X_train.shape,X_test.shape)
    trialstrain.append(X_train)
    labtrain.append(y_train)

    trialstest.append(X_test)
    labtest.append(y_test)


EEG_val=readingEEGdata(df_eeg_val)
X_val=EEG_val.iloc[:, :-1]
y_val=EEG_val.iloc[:,-1:]

for column in X_val.columns:
    #print("test: ",all_min[X_test.columns.get_loc(column)],all_max[X_test.columns.get_loc(column)],X_test[column])
    X_val[column]= c + ((d-c)*(X_val[column] - all_min[X_val.columns.get_loc(column)]) / (all_max[X_val.columns.get_loc(column)] - all_min[X_val.columns.get_loc(column)]))

X_train=pd.concat(trialstrain)
X_test=pd.concat(trialstest)

y_train=pd.concat(labtrain)
y_test=pd.concat(labtest)

EEG_val

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,3900.731464,24763.229614,6542.351646,15559.504669,3543.159960,6541.826672,1502.747624,3968.263654,1696.044642,3172.773266,...,959.927768,376.176957,892.044102,1202.828136,1590.090295,853.048798,2412.075293,669.438435,1136.870742,1
1,5576.712835,15138.536689,4010.441994,15429.920070,2439.457357,2556.268846,2024.594904,3145.382067,1186.792085,1017.547545,...,577.178396,433.383456,434.835192,1009.313744,1224.242858,919.711919,1766.459269,730.661203,2175.817707,1
2,4664.334870,20546.736792,2008.864198,12783.957136,1850.865658,2672.465498,2689.422073,1518.818692,632.232484,1967.194684,...,525.639026,462.347781,1276.411132,781.976712,2415.191466,604.272561,776.670737,978.091561,750.254187,1
3,11574.530321,23894.651028,3215.570563,16406.600522,3240.553864,3439.688708,2686.193287,3224.609235,1064.020423,2723.156222,...,462.878455,496.562388,946.917802,631.466922,2788.392668,907.352642,835.899753,704.194837,482.137494,1
4,11703.169682,19621.243832,6846.210556,23440.774051,7361.669880,6016.996764,2699.282509,3625.704207,2190.582033,4241.510294,...,1105.987752,934.750160,1405.786648,1372.860538,3213.661551,1260.785848,790.565328,758.585978,1702.212663,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,46761.376218,22108.326346,35106.181871,48653.341971,11675.625848,8167.857918,5355.371867,5846.355459,5494.042705,5426.048480,...,2054.511350,666.236735,2226.963675,1004.641813,2333.767324,1607.223094,3256.434193,1749.731053,6288.918422,2
1,51394.245663,40812.723280,31836.320207,17809.385403,9400.935268,12786.795868,4776.797016,3255.061464,3872.186731,6034.669311,...,3153.658733,797.739836,2256.025349,548.597621,2121.000864,941.179147,3610.438508,1524.807822,5220.463676,2
2,11839.608816,48521.805767,4419.211933,28097.480817,2528.795907,14603.937825,3983.166216,3634.411674,1799.514075,6958.897084,...,1356.521327,565.545624,1864.778852,444.513561,3204.597928,616.554990,1160.359016,942.112110,1478.957658,2
3,9353.558544,32076.311650,5133.395119,26723.524390,2567.152773,8330.908605,2814.247611,3978.790818,1341.357519,3838.277536,...,465.083601,316.418673,810.193063,449.551706,1557.722598,832.013387,1022.051520,961.037312,2784.501360,2


# Metodos de Machine Learning:

## k-Nearest Neighbors classifier

In [42]:
#k-Nearest Neighbors classifier
kNN_mdl = KNeighborsClassifier(n_neighbors=3)
kNN_mdl.fit(X_train,y_train)
y_predtrain= kNN_mdl.predict(X_train)
y_predtest= kNN_mdl.predict(X_test)
# printing accuracy
print("Accuracy Train kNN: ",accuracy_score(y_train,y_predtrain))
print("Accuracy Test kNN: ",accuracy_score(y_test,y_predtest))

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Accuracy Train kNN:  1.0
Accuracy Test kNN:  1.0


## Support Vector Machine

In [43]:
#Support Vector Machine
SVM_mdl = svm.SVC(C=5, kernel='poly', gamma='scale', probability=True, random_state=0)
SVM_mdl.fit(X_train, y_train)
y_predtrain= SVM_mdl.predict(X_train)
y_predtest= SVM_mdl.predict(X_test)
# printing accuracy
print("Accuracy Train SVM: ",accuracy_score(y_train,y_predtrain))
print("Accuracy Test SVM: ",accuracy_score(y_test,y_predtest))

Accuracy Train SVM:  1.0
Accuracy Test SVM:  1.0


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Linear Discriminant Analysis

In [44]:
#Linear Discriminant Analysis
LDA_mdl = LinearDiscriminantAnalysis().fit(X_train, y_train)
y_predtrain= LDA_mdl.predict(X_train)
y_predtest= LDA_mdl.predict(X_test)

# printing accuracy
print("Accuracy Train LDA: ",accuracy_score(y_train,y_predtrain))
print("Accuracy Test LDA: ",accuracy_score(y_test,y_predtest))

Accuracy Train LDA:  0.980078125
Accuracy Test LDA:  0.9875


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Decision Tree

In [45]:
#Decision Tree
DT_mdl = DecisionTreeClassifier().fit(X_train, y_train)
y_predtrain= DT_mdl.predict(X_train)
y_predtest= DT_mdl.predict(X_test)

# printing accuracy
print("Accuracy Train DT: ",accuracy_score(y_train,y_predtrain))
print("Accuracy Test DT: ",accuracy_score(y_test,y_predtest))

Accuracy Train DT:  1.0
Accuracy Test DT:  1.0


# Metricas para avaliação:

In [46]:
#Predicted values to evaluate the models generated
y_predval= kNN_mdl.predict(X_val)
print("Accuracy Val kNN: ",accuracy_score(y_val,y_predval))

y_predval= SVM_mdl.predict(X_val)
print("Accuracy Val SVM: ",accuracy_score(y_val,y_predval))

y_predval= LDA_mdl.predict(X_val)
print("Accuracy Val LDA: ",accuracy_score(y_val,y_predval))

y_predval= DT_mdl.predict(X_val)
print("Accuracy Val DT: ",accuracy_score(y_val,y_predval))

Accuracy Val kNN:  0.96875
Accuracy Val SVM:  0.9875
Accuracy Val LDA:  0.95625
Accuracy Val DT:  0.975


# Área de Pratica:

In [47]:
#Split of sessions for train, test and validation
df_eeg_train = pd.concat([sess3,sess2,sess5])
df_eeg_val = pd.concat([sess4,sess1])

#EEG reading data and cross-trials validation
EEG_train = readingEEGdata(df_eeg_train)

In [48]:
# Cria um novo DataFrame (new_df) preenchendo os valores nulos em df_eeg_train com zeros.
new_df = df_eeg_train.fillna(0)

# Seleciona as colunas de índice 2 a 16 de new_df e cria uma cópia em eeg_data.
eeg_data = new_df.iloc[:,2:17].copy()

# Seleciona todas as linhas da coluna de índice 18 de new_df e cria uma lista denominada Id.
Id =  new_df.iloc[:,18].copy()

# Seleciona todas as linhas da coluna de índice 0 de new_df e cria uma lista denominada time.
time = new_df.iloc[:,0].copy()

close_idx = Id.index[Id == 786].tolist() # Identifica os índices em que Id é igual a 786 e os converte em uma lista em close_idx.
open_idx = Id.index[(Id == 769) | (Id == 770)].tolist() # Identifica os índices em que Id é igual a 769 ou 770 e os converte em uma lista em open_idx.
base_idx = Id.index[(Id == 32775) | (Id == 32776)].tolist() # Identifica os índices em que Id é igual a 32775 ou 32776 e os converte em uma lista em base_idx.
end_idx = Id.index[Id == 800].tolist() # Identifica os índices em que Id é igual a 800 e os converte em uma lista em end_idx.

columnlabclase1 = list()
columnlabclase2 = list()
columnlabclassess = list()


print(close_idx)
print("Number of trials: ",len(open_idx))


featclasses = list()
featlab = list()


for i in range(len(open_idx)):
    clase1 = eeg_data.iloc[close_idx[i]:open_idx[i]]
    clase2 = eeg_data.iloc[open_idx[i]:end_idx[i]]

    #Unbalanced class

    clase1 = clase1.iloc[232:695]

    # print("clase1 shape: ",clase1.shape,"clase2 shape: ",clase2.shape)
    # Adding label to each class

    featclase1 = windows_generation_overlap(clase1,1)
    featclase2 = windows_generation_overlap(clase2,2)
    df_classes = pd.concat([featclase1, featclase2], axis=0)
    # print("df_classes: ", df_classes)
    # print("df_classes: ",df_classes)
    featclasses.append(df_classes)


df_totclasses = pd.concat(featclasses, axis=0)

#df_totclasses['labels'] = columnlabclassess
#df_totlabels= pd.concat(featlab, axis=0)
#print("df_totclasses: ",df_totclasses)

df_totclasses

[2087, 3787, 5612, 7312, 9137, 10962, 12662, 14362, 16187, 18012, 19837, 21537, 23362, 25187, 27012, 28712, 2087, 3787, 5612, 7312, 9137, 10962, 12662, 14362, 16187, 18012, 19837, 21537, 23362, 25187, 27012, 28712, 2087, 3787, 5612, 7312, 9137, 10962, 12662, 14362, 16187, 18012, 19837, 21537, 23362, 25187, 27012, 28712]
Number of trials:  48


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,5049.399105,14604.659919,15055.601620,38433.435753,3327.142470,3582.469957,2822.092521,2784.488765,1792.249828,2982.655142,...,485.911859,556.306746,808.305547,1561.199757,1269.041005,1536.520208,1174.416857,1407.845426,742.955862,1
1,10440.460970,15094.812646,9762.699651,16163.799213,3775.185160,5673.840758,4268.703058,2272.253703,2285.652306,3969.024255,...,651.336821,437.814941,929.609811,1180.515939,1595.783915,2334.513627,558.426245,1051.628065,635.797113,1
2,5150.597456,32714.688006,14282.232633,38298.544500,5734.728706,12463.486747,4300.015226,2972.377728,3069.680434,5643.573139,...,1128.462130,543.930415,4595.018150,1104.811727,3974.554069,1761.155155,2591.689523,1180.091085,710.530983,1
3,11026.529380,42695.503382,17752.153089,39472.140967,5408.764714,12298.635822,7238.831630,3026.370696,2988.436743,8060.070770,...,1569.174726,503.890943,6058.569244,1312.889426,4322.873841,1431.608696,4809.379386,1607.942298,2054.202650,1
4,16740.650766,29268.975101,18380.931056,12197.086693,4368.929834,12896.213346,5955.296921,1642.404478,3075.234346,7894.383405,...,618.441923,621.314456,2998.544674,1224.476107,5655.251450,1581.245500,3592.752611,1631.558519,791.718007,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,13779.348034,22098.107817,8612.532015,18234.314909,2384.135893,3759.871193,2436.189555,3549.873717,1471.534100,2910.276733,...,1583.800999,864.321787,1601.161541,1107.059347,1969.088545,1696.330539,2954.595125,1577.454434,4655.157854,2
1,149538.075976,82667.834684,137904.557246,61708.470859,5396.575914,6636.684004,4859.221553,3795.615542,1886.745919,3826.953479,...,7346.359980,3965.621943,6054.088219,2855.351883,5965.925422,1774.810752,13976.735361,3251.251234,5262.540791,2
2,18282.423872,23753.190767,21035.208510,27439.198756,6332.746661,12658.785401,3873.705408,3431.632873,1216.976403,5854.775688,...,2381.326669,869.853338,2400.507105,1206.215671,1838.069340,1023.970142,4335.228535,816.167156,2247.712197,2
3,212865.734753,77196.936607,140725.411439,49157.063127,6314.665074,12514.422093,4407.209683,2066.596837,3729.717172,8126.334960,...,11070.799279,1695.022280,17431.751330,3838.232565,12239.273332,3623.485636,8178.828350,1354.333110,8002.358289,2


In [49]:
# Cria um novo DataFrame (new_df) preenchendo os valores nulos em df_eeg_train com zeros.
new_df = df_eeg_train.fillna(0)

# Seleciona as colunas de índice 2 a 16 de new_df e cria uma cópia em eeg_data.
eeg_data = new_df.iloc[:,2:17].copy()

# Seleciona todas as linhas da coluna de índice 18 de new_df e cria uma lista denominada Id.
Id =  new_df.iloc[:,18].copy()

# Seleciona todas as linhas da coluna de índice 0 de new_df e cria uma lista denominada time.
time = new_df.iloc[:,0].copy()

close_idx = Id.index[Id == 786].tolist() # Identifica os índices em que Id é igual a 786 e os converte em uma lista em close_idx.
open_idx = Id.index[(Id == 769) | (Id == 770)].tolist() # Identifica os índices em que Id é igual a 769 ou 770 e os converte em uma lista em open_idx.
base_idx = Id.index[(Id == 32775) | (Id == 32776)].tolist() # Identifica os índices em que Id é igual a 32775 ou 32776 e os converte em uma lista em base_idx.
end_idx = Id.index[Id == 800].tolist() # Identifica os índices em que Id é igual a 800 e os converte em uma lista em end_idx.

columnlabclase1 = list() # Inicializa listas vazias
columnlabclase2 = list() # Inicializa listas vazias
columnlabclassess = list() # Inicializa listas vazias

print(close_idx)
print("Number of trials: ",len(open_idx))

featclasses = list() # Inicializa listas vazias
featlab = list() # Inicializa listas vazias

# =====================================================================================================
# =====================================================================================================
# =====================================================================================================

for i in range(len(open_idx)):

    clase1 = eeg_data.iloc[close_idx[i]:open_idx[i]]
    clase2 = eeg_data.iloc[open_idx[i]:end_idx[i]]

    # Unbalanced class
    clase1 = clase1.iloc[232:695]

    # print("clase1 shape: ",clase1.shape,"clase2 shape: ",clase2.shape)
    # Adding label to each class
    # feat_clase1 = windows_generation_overlap(clase1,1)
    # Lists to store windows of data

    W = []
    featwind = []
    var = 0

    windows_quantity = int(((clase1.shape[0] - samples) / overlap) - 1)  #Definition of number of windows
    W = [clase1.iloc[var:samples - 1]]
    featwindows = list()

    for i in range(windows_quantity):   #Windowing considering overlap
        var += overlap
        W.append(clase1.iloc[var:(var + samples) - 1])


    # print("# windows: ",windows_quantity,"W: ",len(W),len(W[0]))
    # print("df per window: ",W[0])

    for i in range(len(W)):  # Window 0 to N window

        # print("window n: ",i,len(W))

        df_features = pd.DataFrame(W[i])

        # print("df_features: ",df_features)
        # print("labwind: ",labwind.shape)

        # Raw data organized per window and overlap

        filtered_eegclass = band_pass_filter(df_features, fs_eeg, 8, 30) # Aplicação da Função band_pass_filter

        df_filtered_eegclass = pd.DataFrame(filtered_eegclass)

        CAR_eegclass = CAR_filter(df_filtered_eegclass) # Aplicação da Função CAR_filter

        # print("filtered df_features: ", CAR_eegclass)

        feat_df = features(CAR_eegclass)
        featwind.append(feat_df)
        feat_df[-1] = 1

    df_all = pd.DataFrame(featwind)

    # print("shape all: ",df_all.shape)

    feat_clase2 = windows_generation_overlap(clase2,2)

    df_classes = pd.concat([featclase1, feat_clase2], axis=0)

    # print("df_classes: ", df_classes)
    # print("df_classes: ",df_classes)

    featclasses.append(df_classes)

# =====================================================================================================
# =====================================================================================================
# =====================================================================================================

df_totclasses = pd.concat(featclasses, axis=0)

#df_totclasses['labels'] = columnlabclassess
#df_totlabels= pd.concat(featlab, axis=0)
#print("df_totclasses: ",df_totclasses)

df_totclasses

[2087, 3787, 5612, 7312, 9137, 10962, 12662, 14362, 16187, 18012, 19837, 21537, 23362, 25187, 27012, 28712, 2087, 3787, 5612, 7312, 9137, 10962, 12662, 14362, 16187, 18012, 19837, 21537, 23362, 25187, 27012, 28712, 2087, 3787, 5612, 7312, 9137, 10962, 12662, 14362, 16187, 18012, 19837, 21537, 23362, 25187, 27012, 28712]
Number of trials:  48


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,3900.731464,24763.229614,6542.351646,15559.504669,3543.159960,6541.826672,1502.747624,3968.263654,1696.044642,3172.773266,...,959.927768,376.176957,892.044102,1202.828136,1590.090295,853.048798,2412.075293,669.438435,1136.870742,1
1,5576.712835,15138.536689,4010.441994,15429.920070,2439.457357,2556.268846,2024.594904,3145.382067,1186.792085,1017.547545,...,577.178396,433.383456,434.835192,1009.313744,1224.242858,919.711919,1766.459269,730.661203,2175.817707,1
2,4664.334870,20546.736792,2008.864198,12783.957136,1850.865658,2672.465498,2689.422073,1518.818692,632.232484,1967.194684,...,525.639026,462.347781,1276.411132,781.976712,2415.191466,604.272561,776.670737,978.091561,750.254187,1
3,11574.530321,23894.651028,3215.570563,16406.600522,3240.553864,3439.688708,2686.193287,3224.609235,1064.020423,2723.156222,...,462.878455,496.562388,946.917802,631.466922,2788.392668,907.352642,835.899753,704.194837,482.137494,1
4,11703.169682,19621.243832,6846.210556,23440.774051,7361.669880,6016.996764,2699.282509,3625.704207,2190.582033,4241.510294,...,1105.987752,934.750160,1405.786648,1372.860538,3213.661551,1260.785848,790.565328,758.585978,1702.212663,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,13779.348034,22098.107817,8612.532015,18234.314909,2384.135893,3759.871193,2436.189555,3549.873717,1471.534100,2910.276733,...,1583.800999,864.321787,1601.161541,1107.059347,1969.088545,1696.330539,2954.595125,1577.454434,4655.157854,2
1,149538.075976,82667.834684,137904.557246,61708.470859,5396.575914,6636.684004,4859.221553,3795.615542,1886.745919,3826.953479,...,7346.359980,3965.621943,6054.088219,2855.351883,5965.925422,1774.810752,13976.735361,3251.251234,5262.540791,2
2,18282.423872,23753.190767,21035.208510,27439.198756,6332.746661,12658.785401,3873.705408,3431.632873,1216.976403,5854.775688,...,2381.326669,869.853338,2400.507105,1206.215671,1838.069340,1023.970142,4335.228535,816.167156,2247.712197,2
3,212865.734753,77196.936607,140725.411439,49157.063127,6314.665074,12514.422093,4407.209683,2066.596837,3729.717172,8126.334960,...,11070.799279,1695.022280,17431.751330,3838.232565,12239.273332,3623.485636,8178.828350,1354.333110,8002.358289,2
